# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    PROMPT_KGE_DIMENSION,
    INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    ROOT,
)
from llm_manager import (
    PromptBertClassifier,
)

In [2]:
EPOCHS = 4
BATCH_SIZE_KGE = 128000
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_prompt = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    kge_dimension=PROMPT_KGE_DIMENSION,
    force_recompute=False,
)

loading pretrained model
Device: 'cpu'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [6]:
prompt_embeddings = graph_representation_generator_prompt.get_saved_embeddings("prompt")
save_prompt = False
if prompt_embeddings is None:
    prompt_embeddings = graph_representation_generator_prompt.generate_embeddings(
        kg_manager.llm_df
    )
    save_prompt = True
kg_manager.append_prompt_graph_embeddings(prompt_embeddings, save=save_prompt)


Computing embeddings for embedding dimension 4.


Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [7]:
PROMPT_ROOT = f"{ROOT}/llm/prompt"

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [8]:
prompt_bert_classifier = PromptBertClassifier(
    kg_manager,
    graph_representation_generator_prompt.get_embeddings,
    model_max_length=512,
    root_path=PROMPT_ROOT,
)

Some weights of BertForSequenceClassificationRanges were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We also generate a prompt dataset, this time the prompts also include the KGEs.

In [9]:
dataset_prompt = kg_manager.generate_prompt_embedding_dataset(
    prompt_bert_classifier.tokenize_function,
)

Map:   0%|          | 0/7680049 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200020 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200020 [00:00<?, ? examples/s]

Saving the dataset (0/51 shards):   0%|          | 0/7680049 [00:00<?, ? examples/s]

Saving the dataset (0/22 shards):   0%|          | 0/3200020 [00:00<?, ? examples/s]

Saving the dataset (0/22 shards):   0%|          | 0/3200020 [00:00<?, ? examples/s]

We also train the model. This can be skipped if already done ones.

In [10]:
prompt_bert_classifier.train_model_on_data(
    dataset_prompt, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)

  0%|          | 0/120004 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.7027, 'grad_norm': 1.7521955966949463, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 0.7012, 'grad_norm': 1.4964087009429932, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 0.7009, 'grad_norm': 1.5507583618164062, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 0.6929, 'grad_norm': 1.520392894744873, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6883, 'grad_norm': 0.8891125321388245, 'learning_rate': 5e-06, 'epoch': 0.0}
{'loss': 0.6818, 'grad_norm': 0.7686545848846436, 'learning_rate': 6e-06, 'epoch': 0.0}
{'loss': 0.6764, 'grad_norm': 0.4467780888080597, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6739, 'grad_norm': 0.44365301728248596, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6701, 'grad_norm': 0.3181089758872986, 'learning_rate': 9e-06, 'epoch': 0.0}
{'loss': 0.6722, 'grad_norm': 0.4337005019187927, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 0.6703, 'grad_norm': 0.314503

  0%|          | 0/12501 [00:00<?, ?it/s]

{'eval_loss': 0.11113239079713821, 'eval_accuracy': 0.961630552309048, 'eval_runtime': 3367.135, 'eval_samples_per_second': 950.369, 'eval_steps_per_second': 3.713, 'epoch': 1.0}
{'loss': 0.1406, 'grad_norm': 1.0657668113708496, 'learning_rate': 3.765313294952471e-05, 'epoch': 1.0}
{'loss': 0.1575, 'grad_norm': 1.0500397682189941, 'learning_rate': 3.764894898915518e-05, 'epoch': 1.0}
{'loss': 0.1333, 'grad_norm': 1.018186092376709, 'learning_rate': 3.764476502878565e-05, 'epoch': 1.0}
{'loss': 0.1395, 'grad_norm': 1.3415957689285278, 'learning_rate': 3.764058106841612e-05, 'epoch': 1.0}
{'loss': 0.1433, 'grad_norm': 1.190222978591919, 'learning_rate': 3.76363971080466e-05, 'epoch': 1.0}
{'loss': 0.1444, 'grad_norm': 1.294040560722351, 'learning_rate': 3.763221314767706e-05, 'epoch': 1.0}
{'loss': 0.1501, 'grad_norm': 1.3322794437408447, 'learning_rate': 3.762802918730754e-05, 'epoch': 1.0}
{'loss': 0.1349, 'grad_norm': 0.8667689561843872, 'learning_rate': 3.7623845226938016e-05, 'epoch

...

  0%|          | 0/12501 [00:00<?, ?it/s]

{'eval_loss': 0.09133157879114151, 'eval_accuracy': 0.9711901800613746, 'eval_runtime': 3365.3708, 'eval_samples_per_second': 950.867, 'eval_steps_per_second': 3.715, 'epoch': 3.0}
{'loss': 0.1425, 'grad_norm': 0.876305341720581, 'learning_rate': 1.2549370732360424e-05, 'epoch': 3.0}
{'loss': 0.1266, 'grad_norm': 0.8098748326301575, 'learning_rate': 1.2545186771990897e-05, 'epoch': 3.0}
{'loss': 0.1175, 'grad_norm': 1.0196915864944458, 'learning_rate': 1.2541002811621369e-05, 'epoch': 3.0}
{'loss': 0.1356, 'grad_norm': 1.000313639640808, 'learning_rate': 1.2536818851251841e-05, 'epoch': 3.0}
{'loss': 0.1267, 'grad_norm': 0.8268905878067017, 'learning_rate': 1.2532634890882314e-05, 'epoch': 3.0}
{'loss': 0.1355, 'grad_norm': 0.8722634315490723, 'learning_rate': 1.2528450930512786e-05, 'epoch': 3.0}
{'loss': 0.1238, 'grad_norm': 0.9693925976753235, 'learning_rate': 1.2524266970143259e-05, 'epoch': 3.0}
{'loss': 0.1164, 'grad_norm': 1.0361114740371704, 'learning_rate': 1.2520083009773731e

  0%|          | 0/12501 [00:00<?, ?it/s]

{'eval_loss': 0.0896357074379921, 'eval_accuracy': 0.9719158005262467, 'eval_runtime': 3344.6116, 'eval_samples_per_second': 956.769, 'eval_steps_per_second': 3.738, 'epoch': 4.0}
{'train_runtime': 213703.4046, 'train_samples_per_second': 143.752, 'train_steps_per_second': 0.562, 'train_loss': 0.1433495994062369, 'epoch': 4.0}


AttributeError: 'PromptBertClassifier' object has no attribute 'device'